#### Presentation Types with Hidden Types

* March 2022 version

    * Uses getDistance to identify `close matches` with side-by-side comparison of soggetti.  With a distance of "1", the soggetti `4, 1, 2, 3`, and `5, 1, 2, 3` will count as the same.  These are reported as "flexed entries" in a separate column.

    * Labels Fuga, PEn, and ID according to time intervals.  
    * If two entries are separated by more than 10 bars (80 offsets), the tool resets to a new pattern
    * Finds time intervals between entries (expressed as offsets, like `8.0, 4.0, 8.0`)
    * Finds melodic intervals between first note of successive entries in each pattern (like `P-5, P-8`)
    * Counts number of entries
    * Provides offset and measure/beat locations
    * Sorts all presentation types by the order in which they appear in the piece
    * Reports voice names of the entries, in order of their appearance
    * Omits singleton soggetti (just one entry of a given motive in isolation)
    
    ALSO
    
    * Finds "hidden" types within a longer Fuga.  That is, if a 5-voice fuga also contains a PEN, it will label both of these as separate presentation type, along with all the relevant data noted above.

In [1]:
import intervals
from intervals import * 
from intervals import main_objs
import intervals.visualizations as viz
import pandas as pd
import re
import altair as alt 
from ipywidgets import interact
from pandas.io.json import json_normalize
from pyvis.network import Network
from IPython.display import display
import requests
import os
import numpy
import itertools
MYDIR = ("saved_csv")
CHECK_FOLDER = os.path.isdir(MYDIR)

# If folder doesn't exist, then create it.
if not CHECK_FOLDER:
    os.makedirs(MYDIR)
    print("created folder : ", MYDIR)

else:
    print(MYDIR, "folder already exists.")

saved_csv folder already exists.


#### The following are special functions used by the classifier.  Don't change them.

In [4]:
def find_entry_int_distance(coordinates, piece: intervals.main_objs.ImportedPiece):
    
    """
    This function finds the melodic intervals between the first notes of 
    successive entries in a given presentation type.  
    They are represented as intervals with quality and direction, thus P-4, m3, P5, P5, M-9, P-4, P4
    
    """
    
    tone_list = []
    all_tones = piece.getNoteRest()
    
    for item in coordinates:
        filtered_tones = all_tones.loc[item] 
        tone_list.append(filtered_tones)
        
    noteObjects = [note.Note(tone) for tone in tone_list]
    _ints = [interval.Interval(noteObjects[i], noteObjects[i + 1]) for i in range(len(noteObjects) - 1)]
    entry_ints = []
    
    for _int in _ints:
        entry_ints.append(_int.directedName)
    
    return entry_ints

def split_by_threshold(seq, max_diff=70):  
    
    """
    This function finds gaps between sequences of matching melodic entries.  
    The threshold is set to 70 offsets by default--under about 10 measures.
    
    """
    it = iter(seq)
    last = next(it)
    part = [last]

    for curr in it:
        if curr - last > max_diff:
            yield part
            part = []

        part.append(curr)
        last = curr
#         print(part)
        
    yield part
    

def classify_by_offset(offset_diffs):
    """
    This function predicts the Presentation Types. It relies of the differences between 
    the first offsets of successive melodic entries. 
    
    If the offset differences are identical:  PEN
    If the odd-numbered offset differences are identical:  ID, since these represent
    situations in which the entries 1-2 have the same offset difference as entries 3-4
    If the offset differences are all different:  FUGA
    
    """
    alt_list = offset_diffs[::2]

    if len(set(offset_diffs)) == 1 and len(offset_diffs) > 1:
        return 'PEN'
    # elif (len(offset_difference_list) %2 != 0) and (len(set(alt_list)) == 1):
    elif (len(offset_diffs) % 2 != 0) and (len(set(alt_list)) == 1) and (len(offset_diffs) >= 3):
        return 'ID'
    elif len(offset_diffs) >= 1:
        return 'FUGA'

    

def temp_dict_of_details(slist, entry_array, det, matches):
    """
    This function assembles various features for the presentation types 
    into a single temporary dictionary, which in turn is appended to the dataframe of 'points'
    
    """
    
    array = entry_array[entry_array.index.get_level_values(0).isin(slist)]
    short_offset_list = array.index.to_list()
    time_ints = numpy.diff(array.index).tolist()
    voice_list = array['voice'].to_list()
    tone_coordinates =  list(zip(short_offset_list, voice_list))
    mel_ints = find_entry_int_distance(tone_coordinates, piece)
    first_offset = short_offset_list[0]
    meas_beat = det[det.index.get_level_values('Offset').isin(short_offset_list)]
    mb2 = meas_beat.reset_index()
    mb2['mb'] = mb2["Measure"].astype(str) + "/" + mb2["Beat"].astype(str)
    meas_beat_list = mb2['mb'].to_list()
    
    # temp results for this set
    temp = {"Composer": piece.metadata["composer"],
            "Title": piece.metadata["title"],
            'First_Offset': first_offset, 
            'Offsets': short_offset_list, 
            'Measures_Beats': meas_beat_list,
            "Soggetti": matches,
            'Voices': voice_list, 
            'Time_Entry_Intervals': time_ints, 
            'Melodic_Entry_Intervals': mel_ints}
    return temp

def classify_entries_as_presentation_types(piece):
    
    """
    This function uses several other functions to classify the entries in a given piece.
    The output is a list, in order of offset, of each presentation type, including information about
    measures/beats
    starting offset
    soggetti involved 
    melodic intervals of entry
    time intervals of entry
    
    
    """
    # Classifier with Functions
    points = pd.DataFrame()
    points2 = pd.DataFrame()
    # new_offset_list = []
    nr = piece.getNoteRest()
    det = piece.detailIndex(nr, offset=True)

    # durations and ngrams of durations
    dur = piece.getDuration(df=nr)
    dur_ng = piece.getNgrams(df=dur, n=4)

    # ngrams of melodic entries
    # for chromatic, use:
    # piece.getMelodicEntries(interval_settings=('c', True, True), n=5)
    mel = piece.getMelodicEntries(n=4)
    mels_stacked = mel.stack().to_frame()
    mels_stacked.rename(columns =  {0:"pattern"}, inplace = True)

    # edit distance, based on side-by-side comparison of melodic ngrams
    # gets flexed and other similar soggetti
    dist = piece.getDistance(mel)
    dist_stack = dist.stack().to_frame()


    # filter distances to threshold.  <2 is good
    filtered_dist_stack = dist_stack[dist_stack[0] < 2]
    filtered_dist = filtered_dist_stack.reset_index()
    filtered_dist.rename(columns =  {'level_0':"source", 'level_1':'match'}, inplace = True)

    # Group the filtered distanced patterns
    full_list_of_matches = filtered_dist.groupby('source')['match'].apply(list).reset_index()

    for matches in full_list_of_matches["match"]:
        related_entry_list = mels_stacked[mels_stacked['pattern'].isin(matches)]
        entry_array = related_entry_list.reset_index(level=1).rename(columns = {'level_1': "voice", 0: "pattern"})
        offset_list = entry_array.index.to_list()
        split_list = list(split_by_threshold(offset_list))
        # here is the list of starting offsets of the original set of entries:  slist
        slist = split_list[0]
        temp = temp_dict_of_details(slist, entry_array, det, matches)

        points = points.append(temp, ignore_index=True)
        points['Presentation_Type'] = points['Time_Entry_Intervals'].apply(classify_by_offset)
        points.drop_duplicates(subset=["First_Offset"], keep='first', inplace = True)
        points = points[points['Offsets'].apply(len) > 1]

        l = len(slist)
        if l > 2:
            for r in range(3, l):
    #             list_combinations = list(combinations(slist, r))
                list_combinations = list(combinations(slist, r))
                for slist in list_combinations:

                    temp = temp_dict_of_details(slist, entry_array, det, matches)

                    temp["Presentation_Type"] = classify_by_offset(temp['Time_Entry_Intervals'])

                    if 'PEN' in temp["Presentation_Type"]:
                        points2 = points2.append(temp, ignore_index=True)#.sort_values("First_Offset")
    #                     points = points.append(combo_temp, ignore_index=True).sort_values("First_Offset")
                        points2 = points2[points2['Offsets'].apply(len) > 1]
                    if 'ID' in temp["Presentation_Type"]:
                        points2 = points2.append(combo_temp, ignore_index=True)#.sort_values("First_Offset")
    #                     points = points.append(combo_temp, ignore_index=True).sort_values("First_Offset")
#                 points2.sort_values("First_Offset")
                points2.drop_duplicates(subset=["First_Offset"], keep='first', inplace = True)

    points_combined = points.append(points2, ignore_index=True).sort_values("First_Offset").reset_index(drop=True)
    points_combined['Flexed_Entries'] = points_combined["Soggetti"].apply(len) > 1
    points_combined["Number_Entries"] = points_combined["Offsets"].apply(len) 
    col_order = ['Composer', 
             'Title', 
             'First_Offset', 
             'Measures_Beats', 
             'Melodic_Entry_Intervals',
             'Offsets', 
             'Soggetti', 
             'Time_Entry_Intervals', 
             'Voices',
             'Presentation_Type', 
              'Number_Entries',
            'Flexed_Entries']
    points_combined = points_combined.reindex(columns=col_order)
    return points_combined

In [5]:
offset_diffs = [2.0, 1.0, 2.0, 3.0, 5.0, 6.0]
# some_list[start:stop:step]
alt_list = offset_diffs[::2]
alt_list

[2.0, 2.0, 5.0]

In [6]:
if len(set(offset_diffs)) == 1 and len(offset_diffs) > 1:
    print('This is a PEN')
    # elif (len(offset_difference_list) %2 != 0) and (len(set(alt_list)) == 1):
elif (len(offset_diffs) % 2 != 0) and (len(set(alt_list)) == 1) and (len(offset_diffs) >= 3):
    print('This is an ID')
elif len(offset_diffs) >= 1:
    print('This is a FUGA')

This is a FUGA


In [7]:
l = len(offset_diffs)
# print(l)
if l > 2:
    for r in range(3, l):
        print(r)
        list_combinations = list(combinations(offset_diffs, r))
#         for slist in list_combinations:
        print(list_combinations)


3
[(2.0, 1.0, 2.0), (2.0, 1.0, 3.0), (2.0, 1.0, 5.0), (2.0, 1.0, 6.0), (2.0, 2.0, 3.0), (2.0, 2.0, 5.0), (2.0, 2.0, 6.0), (2.0, 3.0, 5.0), (2.0, 3.0, 6.0), (2.0, 5.0, 6.0), (1.0, 2.0, 3.0), (1.0, 2.0, 5.0), (1.0, 2.0, 6.0), (1.0, 3.0, 5.0), (1.0, 3.0, 6.0), (1.0, 5.0, 6.0), (2.0, 3.0, 5.0), (2.0, 3.0, 6.0), (2.0, 5.0, 6.0), (3.0, 5.0, 6.0)]
4
[(2.0, 1.0, 2.0, 3.0), (2.0, 1.0, 2.0, 5.0), (2.0, 1.0, 2.0, 6.0), (2.0, 1.0, 3.0, 5.0), (2.0, 1.0, 3.0, 6.0), (2.0, 1.0, 5.0, 6.0), (2.0, 2.0, 3.0, 5.0), (2.0, 2.0, 3.0, 6.0), (2.0, 2.0, 5.0, 6.0), (2.0, 3.0, 5.0, 6.0), (1.0, 2.0, 3.0, 5.0), (1.0, 2.0, 3.0, 6.0), (1.0, 2.0, 5.0, 6.0), (1.0, 3.0, 5.0, 6.0), (2.0, 3.0, 5.0, 6.0)]
5
[(2.0, 1.0, 2.0, 3.0, 5.0), (2.0, 1.0, 2.0, 3.0, 6.0), (2.0, 1.0, 2.0, 5.0, 6.0), (2.0, 1.0, 3.0, 5.0, 6.0), (2.0, 2.0, 3.0, 5.0, 6.0), (1.0, 2.0, 3.0, 5.0, 6.0)]


## Load the Piece Here

* Note that you can load from CRIM, or put a file in the **Music_Files** folder in the Notebook.

In [63]:
# piece = importScore('Music_Files/Senfl_Ave_forCRIM.mei_msg.mei')
piece = importScore('https://crimproject.org/mei/CRIM_Mass_0013_3.mei')
# piece = importScore('Music_Files/CRIM_Mass_0007_4.mei')


Memoized piece detected.


## Run the Classifier Here.  



In [64]:
output = classify_entries_as_presentation_types(piece)
output

,Composer,Title,First_Offset,Measures_Beats,Melodic_Entry_Intervals,Offsets,Soggetti,Time_Entry_Intervals,Voices,Presentation_Type,Number_Entries,Flexed_Entries
0,Pierre de Manchicourt,Missa Quo abiit dilectus tuus: Credo,0.0,"[1/1.0, 3/1.0, 8/3.0, 10/1.0]","[P-4, P-5, P-4]","[0.0, 16.0, 60.0, 72.0]","[5, -2, 2, 2, 4, -2, 2, 2]","[16.0, 44.0, 12.0]","[Superius, Contratenor, Tenor, Bassus]",FUGA,4,True
1,Pierre de Manchicourt,Missa Quo abiit dilectus tuus: Credo,38.0,"[5/4.0, 7/2.0, 14/4.0, 15/4.0, 16/2.0, 18/2.0]","[P-5, M2, P-5, P-4, P11]","[38.0, 50.0, 110.0, 118.0, 122.0, 138.0]","[-3, 2, -2, -2, -3, 2, -2, -3]","[12.0, 60.0, 8.0, 4.0, 16.0]","[Superius, Contratenor, Superius, Tenor, Bassu...",FUGA,6,True
2,Pierre de Manchicourt,Missa Quo abiit dilectus tuus: Credo,94.0,"[12/4.0, 13/4.0, 17/2.0]","[P4, M2]","[94.0, 102.0, 130.0]","[-3, 2, -2, 2, -3, 2, -3, 2]","[8.0, 28.0]","[Tenor, Contratenor, Contratenor]",FUGA,3,True
3,Pierre de Manchicourt,Missa Quo abiit dilectus tuus: Credo,162.0,"[21/2.0, 22/4.0]",[P5],"[162.0, 174.0]","[1, 3, -2, -2]",[12.0],"[Contratenor, Superius]",FUGA,2,False
4,Pierre de Manchicourt,Missa Quo abiit dilectus tuus: Credo,198.0,"[25/4.0, 27/2.0]",[P5],"[198.0, 210.0]","[3, -2, -2, -2, 1, -2, -2, -2, 2, -2, -2, -2]",[12.0],"[Bassus, Tenor]",FUGA,2,True
5,Pierre de Manchicourt,Missa Quo abiit dilectus tuus: Credo,222.0,"[28/4.0, 30/4.0, 35/4.0, 36/4.0, 38/2.0]","[P4, P-8, P8, P-8]","[222.0, 238.0, 278.0, 286.0, 298.0]","[2, -3, 5, 1]","[16.0, 40.0, 8.0, 12.0]","[Contratenor, Superius, Contratenor, Superius,...",FUGA,5,False
6,Pierre de Manchicourt,Missa Quo abiit dilectus tuus: Credo,258.0,"[33/2.0, 34/2.0]",[P8],"[258.0, 266.0]","[2, -3, 5, -5]",[8.0],"[Bassus, Tenor]",FUGA,2,False
7,Pierre de Manchicourt,Missa Quo abiit dilectus tuus: Credo,314.0,"[40/2.0, 40/4.0, 41/4.0, 44/4.0]","[P4, P-8, P12]","[314.0, 318.0, 326.0, 350.0]","[1, 1, 2, 1, 1, 1, 2, 2]","[4.0, 8.0, 24.0]","[Tenor, Contratenor, Bassus, Superius]",FUGA,4,True
8,Pierre de Manchicourt,Missa Quo abiit dilectus tuus: Credo,330.0,"[42/2.0, 43/4.0, 47/4.0, 49/2.0, 49/4.0]","[P-5, P8, P-4, P4]","[330.0, 342.0, 374.0, 386.0, 390.0]","[1, -2, -2, -2, 2, -2, -2, -2]","[12.0, 32.0, 12.0, 4.0]","[Tenor, Bassus, Tenor, Bassus, Contratenor]",FUGA,5,True
9,Pierre de Manchicourt,Missa Quo abiit dilectus tuus: Credo,410.0,"[52/2.0, 53/2.0, 54/4.0, 55/4.0, 56/4.0, 57/4....","[P-5, P-4, P-5, P8, P1, P-5]","[410.0, 418.0, 430.0, 438.0, 446.0, 454.0, 462.0]","[4, 2, 2, -3, 5, 2, 2, -3]","[8.0, 12.0, 8.0, 8.0, 8.0, 8.0]","[Superius, Contratenor, Tenor, Bassus, Contrat...",FUGA,7,True


In [113]:
def joiner(a):
    b = '_'.join(map(str, a))
    return b
def clean_str(c):  
    re.sub(r'\W+', '', c)
    return c
def clean_str2(c):  
    d = (('_').join(c[0].split(","))).replace(" ", "")
#     d = d.replace(" ", "")
    return d


In [114]:
g = ['3, 3, 1, -2', '3, 4, 1, -2']
clean_str2(g)

3, 3, 1, -2
<class 'str'>


'3_3_1_-2'

In [77]:
a = output["Soggetti"][23]
clean_str(a)

TypeError: expected string or bytes-like object

In [12]:
piece_list = ['https://crimproject.org/mei/CRIM_Model_0008.mei',
             'https://crimproject.org/mei/CRIM_Mass_0005_1.mei',
             'https://crimproject.org/mei/CRIM_Mass_0005_2.mei',
             'https://crimproject.org/mei/CRIM_Mass_0005_3.mei',
             'https://crimproject.org/mei/CRIM_Mass_0005_4.mei',
             'https://crimproject.org/mei/CRIM_Mass_0005_5.mei'
             'https://crimproject.org/mei/CRIM_Model_0001.mei',
             'https://crimproject.org/mei/CRIM_Mass_0002_1.mei',
             'https://crimproject.org/mei/CRIM_Mass_0002_2.mei',
             'https://crimproject.org/mei/CRIM_Mass_0002_3.mei',
             'https://crimproject.org/mei/CRIM_Mass_0002_4.mei',
             'https://crimproject.org/mei/CRIM_Mass_0002_5.mei',
             'https://crimproject.org/mei/CRIM_Model_0015.mei',
             'https://crimproject.org/mei/CRIM_Mass_0013_1.mei',
             'https://crimproject.org/mei/CRIM_Mass_0013_2.mei',
             'https://crimproject.org/mei/CRIM_Mass_0013_3.mei',
             'https://crimproject.org/mei/CRIM_Mass_0013_4.mei',
             'https://crimproject.org/mei/CRIM_Mass_0013_5.mei',
             'https://crimproject.org/mei/CRIM_Model_0019.mei',
             'https://crimproject.org/mei/CRIM_Mass_0019_1.mei',
             'https://crimproject.org/mei/CRIM_Mass_0019_2.mei',
             'https://crimproject.org/mei/CRIM_Mass_0019_3.mei',
             'https://crimproject.org/mei/CRIM_Mass_0019_4.mei',
             'https://crimproject.org/mei/CRIM_Mass_0019_5.mei',]

# piece_list = ['https://crimproject.org/mei/CRIM_Model_0008.mei']

In [62]:
models = ['https://crimproject.org/mei/CRIM_Model_0008.mei',
             'https://crimproject.org/mei/CRIM_Model_0001.mei',
             'https://crimproject.org/mei/CRIM_Model_0015.mei',
             'https://crimproject.org/mei/CRIM_Model_0019.mei']

In [49]:
final = pd.DataFrame()
for work in piece_list:
    piece = importScore(work)
    output = classify_entries_as_presentation_types(piece)
#     print(output)
    final = final.append(output, ignore_index=True)
# #     final.append(points_combined)
final

Memoized piece detected.
Successfully imported https://crimproject.org/mei/CRIM_Model_0001.mei
Successfully imported https://crimproject.org/mei/CRIM_Model_0015.mei
Successfully imported https://crimproject.org/mei/CRIM_Model_0019.mei


,Composer,Title,First_Offset,Measures_Beats,Melodic_Entry_Intervals,Offsets,Soggetti,Time_Entry_Intervals,Voices,Presentation_Type,Number_Entries,Flexed_Entries
0,Josquin Des Prés,Ave Maria,0.0,"[1/1.0, 3/1.0, 5/1.0, 7/1.0]","[P-8, P1, P-8]","[0.0, 16.0, 32.0, 48.0]","[4, 1, 2, 2]","[16.0, 16.0, 16.0]","[[Superius], Altus, Tenor, Bassus]",PEN,4,False
1,Josquin Des Prés,Ave Maria,0.0,"[1/1.0, 3/1.0, 5/1.0]","[P-8, P1]","[0.0, 16.0, 32.0]","[4, 1, 2, 2]","[16.0, 16.0]","[[Superius], Altus, Tenor]",PEN,3,False
2,Josquin Des Prés,Ave Maria,16.0,"[3/1.0, 5/1.0, 7/1.0]","[P1, P-8]","[16.0, 32.0, 48.0]","[4, 1, 2, 2]","[16.0, 16.0]","[Altus, Tenor, Bassus]",PEN,3,False
3,Josquin Des Prés,Ave Maria,56.0,"[8/1.0, 10/1.0, 12/1.0]","[P-8, P1]","[56.0, 72.0, 88.0]","[-2, -2, -2, 2]","[16.0, 16.0]","[[Superius], Altus, Tenor]",PEN,3,False
4,Josquin Des Prés,Ave Maria,56.0,"[8/1.0, 10/1.0, 12/1.0, 14/1.0]","[P-8, P1, P-8]","[56.0, 72.0, 88.0, 104.0]","[-2, -2, -2, 2]","[16.0, 16.0, 16.0]","[[Superius], Altus, Tenor, Bassus]",PEN,4,False
...,...,...,...,...,...,...,...,...,...,...,...,...
82,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,298.0,"[38/2.0, 38/4.0, 40/2.0, 42/2.0, 44/1.0, 46/2....","[P-5, P8, P-12, P8, P8, P-11]","[298.0, 302.0, 314.0, 330.0, 344.0, 362.0, 382.0]","[1, 1, -3, 3]","[4.0, 12.0, 16.0, 14.0, 18.0, 20.0]","[Altus, Tenor, Cantus, Bassus, Altus, Cantus, ...",FUGA,7,False
83,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,298.0,"[38/2.0, 40/2.0, 42/2.0]","[P4, P-12]","[298.0, 314.0, 330.0]","[1, 1, -3, 3]","[16.0, 16.0]","[Altus, Cantus, Bassus]",PEN,3,False
84,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,394.0,"[50/2.0, 51/4.0, 53/2.0, 55/2.0, 56/4.0, 59/1....","[P5, P-8, P4, P5, P-8, P8, P-4, P-5]","[394.0, 406.0, 418.0, 434.0, 446.0, 464.0, 486...","[-2, 2, 1, 3]","[12.0, 12.0, 16.0, 12.0, 18.0, 22.0, 4.0, 16.0]","[Altus, Cantus, Bassus, Tenor, Altus, Bassus, ...",FUGA,9,False
85,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,394.0,"[50/2.0, 51/4.0, 53/2.0]","[P5, P-8]","[394.0, 406.0, 418.0]","[-2, 2, 1, 3]","[12.0, 12.0]","[Altus, Cantus, Bassus]",PEN,3,False


In [14]:
final.tail()

,Composer,Title,First_Offset,Measures_Beats,Melodic_Entry_Intervals,Offsets,Soggetti,Time_Entry_Intervals,Voices,Presentation_Type,Number_Entries,Flexed_Entries
14,Josquin Des Prés,Ave Maria,616.0,"[78/1.0, 81/1.0]",[P-8],"[616.0, 640.0]","[1, 1, 5, 1]",[24.0],"[Altus, Bassus]",FUGA,2,False
15,Josquin Des Prés,Ave Maria,944.0,"[111/1.0, 115/1.0, 119/1.0, 123/1.0]","[P-8, P8, P-8]","[944.0, 972.0, 1008.0, 1036.0]","[1, -2, -2, -2, 1, -2, -2, -3]","[28.0, 36.0, 28.0]","[Altus, Bassus, Altus, Bassus]",ID,4,True
16,Josquin Des Prés,Ave Maria,976.0,"[115/1.0, 123/1.0]",[P1],"[976.0, 1040.0]","[2, 2, 1, 2]",[64.0],"[Altus, Altus]",FUGA,2,False
17,Josquin Des Prés,Ave Maria,1072.0,"[127/1.0, 130/1.0]",[P-8],"[1072.0, 1092.0]","[2, 2, 2, -2]",[20.0],"[[Superius], Tenor]",FUGA,2,False
18,Josquin Des Prés,Ave Maria,1120.0,"[133/1.0, 137/1.0]",[P-8],"[1120.0, 1148.0]","[1, -2, -2, 2]",[28.0],"[[Superius], Tenor]",FUGA,2,False


In [66]:
final["Soggetti"][15][0]

'1, -2, -2, -2'

In [50]:
final["MINT"] = final["Melodic_Entry_Intervals"].apply(joiner)
final["TINT"] = final["Time_Entry_Intervals"].apply(joiner)
# final['SOG'] = final["Soggetti"][0]
# final['SOG2'] = final['SOG'].apply(joiner)
# final['SOG3'] = final['SOG2'].apply(clean_str)
final["ALL_INT"] = final["MINT"] + '_' + final["TINT"]
# final["ALL_SOG"] = final["MINT"] + '_' + final["SOG2"]
final

,Composer,Title,First_Offset,Measures_Beats,Melodic_Entry_Intervals,Offsets,Soggetti,Time_Entry_Intervals,Voices,Presentation_Type,Number_Entries,Flexed_Entries,MINT,TINT,ALL_INT
0,Josquin Des Prés,Ave Maria,0.0,"[1/1.0, 3/1.0, 5/1.0, 7/1.0]","[P-8, P1, P-8]","[0.0, 16.0, 32.0, 48.0]","[4, 1, 2, 2]","[16.0, 16.0, 16.0]","[[Superius], Altus, Tenor, Bassus]",PEN,4,False,P-8_P1_P-8,16.0_16.0_16.0,P-8_P1_P-8_16.0_16.0_16.0
1,Josquin Des Prés,Ave Maria,0.0,"[1/1.0, 3/1.0, 5/1.0]","[P-8, P1]","[0.0, 16.0, 32.0]","[4, 1, 2, 2]","[16.0, 16.0]","[[Superius], Altus, Tenor]",PEN,3,False,P-8_P1,16.0_16.0,P-8_P1_16.0_16.0
2,Josquin Des Prés,Ave Maria,16.0,"[3/1.0, 5/1.0, 7/1.0]","[P1, P-8]","[16.0, 32.0, 48.0]","[4, 1, 2, 2]","[16.0, 16.0]","[Altus, Tenor, Bassus]",PEN,3,False,P1_P-8,16.0_16.0,P1_P-8_16.0_16.0
3,Josquin Des Prés,Ave Maria,56.0,"[8/1.0, 10/1.0, 12/1.0]","[P-8, P1]","[56.0, 72.0, 88.0]","[-2, -2, -2, 2]","[16.0, 16.0]","[[Superius], Altus, Tenor]",PEN,3,False,P-8_P1,16.0_16.0,P-8_P1_16.0_16.0
4,Josquin Des Prés,Ave Maria,56.0,"[8/1.0, 10/1.0, 12/1.0, 14/1.0]","[P-8, P1, P-8]","[56.0, 72.0, 88.0, 104.0]","[-2, -2, -2, 2]","[16.0, 16.0, 16.0]","[[Superius], Altus, Tenor, Bassus]",PEN,4,False,P-8_P1_P-8,16.0_16.0_16.0,P-8_P1_P-8_16.0_16.0_16.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,298.0,"[38/2.0, 38/4.0, 40/2.0, 42/2.0, 44/1.0, 46/2....","[P-5, P8, P-12, P8, P8, P-11]","[298.0, 302.0, 314.0, 330.0, 344.0, 362.0, 382.0]","[1, 1, -3, 3]","[4.0, 12.0, 16.0, 14.0, 18.0, 20.0]","[Altus, Tenor, Cantus, Bassus, Altus, Cantus, ...",FUGA,7,False,P-5_P8_P-12_P8_P8_P-11,4.0_12.0_16.0_14.0_18.0_20.0,P-5_P8_P-12_P8_P8_P-11_4.0_12.0_16.0_14.0_18.0...
83,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,298.0,"[38/2.0, 40/2.0, 42/2.0]","[P4, P-12]","[298.0, 314.0, 330.0]","[1, 1, -3, 3]","[16.0, 16.0]","[Altus, Cantus, Bassus]",PEN,3,False,P4_P-12,16.0_16.0,P4_P-12_16.0_16.0
84,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,394.0,"[50/2.0, 51/4.0, 53/2.0, 55/2.0, 56/4.0, 59/1....","[P5, P-8, P4, P5, P-8, P8, P-4, P-5]","[394.0, 406.0, 418.0, 434.0, 446.0, 464.0, 486...","[-2, 2, 1, 3]","[12.0, 12.0, 16.0, 12.0, 18.0, 22.0, 4.0, 16.0]","[Altus, Cantus, Bassus, Tenor, Altus, Bassus, ...",FUGA,9,False,P5_P-8_P4_P5_P-8_P8_P-4_P-5,12.0_12.0_16.0_12.0_18.0_22.0_4.0_16.0,P5_P-8_P4_P5_P-8_P8_P-4_P-5_12.0_12.0_16.0_12....
85,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,394.0,"[50/2.0, 51/4.0, 53/2.0]","[P5, P-8]","[394.0, 406.0, 418.0]","[-2, 2, 1, 3]","[12.0, 12.0]","[Altus, Cantus, Bassus]",PEN,3,False,P5_P-8,12.0_12.0,P5_P-8_12.0_12.0


In [16]:
b = final["Soggetti"][0]
v = joiner_sog(b)
type(v)

NameError: name 'joiner_sog' is not defined

In [17]:
e = final["SOG2"][0]
re.sub(r'\W+', '', e)
# type(e)

KeyError: 'SOG2'

In [51]:
filtered = final.loc[final['Number_Entries'] < 4] 
filtered

,Composer,Title,First_Offset,Measures_Beats,Melodic_Entry_Intervals,Offsets,Soggetti,Time_Entry_Intervals,Voices,Presentation_Type,Number_Entries,Flexed_Entries,MINT,TINT,ALL_INT
1,Josquin Des Prés,Ave Maria,0.0,"[1/1.0, 3/1.0, 5/1.0]","[P-8, P1]","[0.0, 16.0, 32.0]","[4, 1, 2, 2]","[16.0, 16.0]","[[Superius], Altus, Tenor]",PEN,3,False,P-8_P1,16.0_16.0,P-8_P1_16.0_16.0
2,Josquin Des Prés,Ave Maria,16.0,"[3/1.0, 5/1.0, 7/1.0]","[P1, P-8]","[16.0, 32.0, 48.0]","[4, 1, 2, 2]","[16.0, 16.0]","[Altus, Tenor, Bassus]",PEN,3,False,P1_P-8,16.0_16.0,P1_P-8_16.0_16.0
3,Josquin Des Prés,Ave Maria,56.0,"[8/1.0, 10/1.0, 12/1.0]","[P-8, P1]","[56.0, 72.0, 88.0]","[-2, -2, -2, 2]","[16.0, 16.0]","[[Superius], Altus, Tenor]",PEN,3,False,P-8_P1,16.0_16.0,P-8_P1_16.0_16.0
5,Josquin Des Prés,Ave Maria,72.0,"[10/1.0, 12/1.0, 14/1.0]","[P1, P-8]","[72.0, 88.0, 104.0]","[-2, -2, -2, 2]","[16.0, 16.0]","[Altus, Tenor, Bassus]",PEN,3,False,P1_P-8,16.0_16.0,P1_P-8_16.0_16.0
6,Josquin Des Prés,Ave Maria,124.0,"[16/3.0, 18/3.0, 20/3.0]","[P-8, P1]","[124.0, 140.0, 156.0]","[1, 1, 2, 2, 1, 1, 1, 2]","[16.0, 16.0]","[[Superius], Altus, Tenor]",PEN,3,True,P-8_P1,16.0_16.0,P-8_P1_16.0_16.0
8,Josquin Des Prés,Ave Maria,140.0,"[18/3.0, 20/3.0, 22/3.0]","[P1, P-8]","[140.0, 156.0, 172.0]","[1, 1, 2, 2, 1, 1, 1, 2]","[16.0, 16.0]","[Altus, Tenor, Bassus]",PEN,3,True,P1_P-8,16.0_16.0,P1_P-8_16.0_16.0
9,Josquin Des Prés,Ave Maria,176.0,"[23/1.0, 28/1.0]",[P-8],"[176.0, 216.0]","[-3, -2, 2, 2]",[40.0],"[Altus, Bassus]",FUGA,2,False,P-8,40.0,P-8_40.0
10,Josquin Des Prés,Ave Maria,244.0,"[31/3.0, 35/3.0]",[P-8],"[244.0, 276.0]","[1, 2, 2, 1, 1, 1, 2, 1]",[32.0],"[[Superius], Tenor]",FUGA,2,True,P-8,32.0,P-8_32.0
11,Josquin Des Prés,Ave Maria,316.0,[40/3.0],[M-6],"[316.0, 316.0]","[1, 1, 2, -2, 1, 1, 2, -3]",[0.0],"[[Superius], Tenor]",FUGA,2,True,M-6,0.0,M-6_0.0
14,Josquin Des Prés,Ave Maria,616.0,"[78/1.0, 81/1.0]",[P-8],"[616.0, 640.0]","[1, 1, 5, 1]",[24.0],"[Altus, Bassus]",FUGA,2,False,P-8,24.0,P-8_24.0


In [52]:
# f2 = filtered.groupby('MINT').count.reset_index()
f2 = filtered.groupby('MINT')['Title'].apply(list).reset_index()
f2

,MINT,Title
0,M-2_P1,[Veni speciosam]
1,M-2_P5,[Veni speciosam]
2,M-6,[Ave Maria]
3,M-9_P1,[Quo abiit dilectus tuus]
4,M2_P-5,[Quo abiit dilectus tuus]
5,P-4,[Veni speciosam]
6,P-4_P-5,[Quo abiit dilectus tuus]
7,P-5_M9,[Quo abiit dilectus tuus]
8,P-5_P-5,"[Veni speciosam, Quo abiit dilectus tuus]"
9,P-5_P1,[Quo abiit dilectus tuus]


In [53]:
pairs = f2.Title.apply(lambda x: list(combinations(x, 2)))
pairs

0                                                    []
1                                                    []
2                                                    []
3                                                    []
4                                                    []
5                                                    []
6                                                    []
7                                                    []
8           [(Veni speciosam, Quo abiit dilectus tuus)]
9                                                    []
10                                                   []
11                                                   []
12    [(Ave Maria, Ave Maria), (Ave Maria, Ave Maria...
13                                                   []
14                                                   []
15    [(Ave Maria, Ave Maria), (Ave Maria, Ave Maria...
16    [(Veni speciosam, Quo abiit dilectus tuus), (V...
17                        [(Ave Maria, Veni spec

In [54]:
unique_pairs = pairs.explode().dropna().unique()
unique_pairs

array([('Veni speciosam', 'Quo abiit dilectus tuus'),
       ('Ave Maria', 'Ave Maria'),
       ('Ave Maria', 'Quo abiit dilectus tuus'),
       ('Veni speciosam', 'Veni sponsa Christi'),
       ('Quo abiit dilectus tuus', 'Veni sponsa Christi'),
       ('Ave Maria', 'Veni speciosam'),
       ('Veni sponsa Christi', 'Veni sponsa Christi'),
       ('Veni speciosam', 'Veni speciosam')], dtype=object)

In [55]:
pd.Series(unique_pairs).isna().sum()

0

In [56]:
from pyvis.network import Network
from itertools import combinations
import networkx as nx
import re
from community import community_louvain
from copy import deepcopy

In [57]:
def add_communities(G):
    G = deepcopy(G)
    partition = community_louvain.best_partition(G)
    nx.set_node_attributes(G, partition, "group")
    return G

In [58]:
pyvis_graph = Network(notebook=False, width="1800", height="1400", bgcolor="black", font_color="white")

In [59]:
G = nx.Graph()
G.add_edges_from(unique_pairs)
G = add_communities(G)

In [60]:
pyvis_graph.from_nx(G)

In [61]:
pyvis_graph.show("MINT.html")

In [ ]:
output = output.loc[output['Presentation_Type'] == "PEN"] 
output

In [ ]:
filtered = output.loc[output['Number_Entries'] < 4] 
filtered

#### Below is Development Work

In [ ]:
# this works with ONE list of offsets

points2 = pd.DataFrame()
split_list = [90.0, 94.0, 102.0, 106.0, 134.0, 146.0, 162.0]

l = len(split_list)  
for r in range(3, l):
    list_combinations = list(combinations(split_list, r))
#             combo_time_ints = []
    for combo in list_combinations:
        combo_time_ints = numpy.diff(combo).tolist()
        combo_array = entry_array[entry_array.index.get_level_values(0).isin(combo)]
        combo_voice_list = combo_array['voice'].to_list()
        combo_patterns = combo_array['pattern']
        unique_combo_patterns = list(set(combo_patterns))
        tone_coordinates =  list(zip(combo, combo_voice_list))
# tone_coordinates.ffill(inplace=True)
        mel_ints = find_entry_int_distance(tone_coordinates, piece)
        hidden_type = classify_by_offset(combo_time_ints)

        meas_beat = det[det.index.get_level_values('Offset').isin(combo)]
        mb2 = meas_beat.reset_index()
        mb2['mb'] = mb2["Measure"].astype(str) + "/" + mb2["Beat"].astype(str)
        meas_beat_list = mb2['mb'].to_list()

        combo_temp = {'First_Offset': combo[0], 
            'Offsets': combo, 
            'Measures_Beats': meas_beat_list,
            'Presentation_Type': hidden_type,
            "Soggetti": unique_combo_patterns,
            'Voices': combo_voice_list, 
            'Time_Entry_Intervals': combo_time_ints, 
            'Melodic_Entry_Intervals': mel_ints}

        if 'PEN' in hidden_type:
            points2 = points2.append(combo_temp, ignore_index=True).sort_values("First_Offset")
#             points2 = points2[points2['Offsets'].apply(len) > 1]
        if 'ID' in hidden_type:
            points2 = points2.append(combo_temp, ignore_index=True).sort_values("First_Offset")
#             points2 = points2[points2['Offsets'].apply(len) > 1]
        
        
# combo_time_ints
# combo_array
# # combo_voice_list
# # combo_patterns
# # unique_combo_patterns
# # tone_coordinates
# # mel_ints
# # combo_temp
points2

In [ ]:
# this finds hidden fugas.  
# try to run each of the first set of results above ('points') through this tool, then append the 
# new results to the full DF, and sort again.  
# mark each long pattern with 'has hidden pattern' boolean?  or ?

sample_list = points["Offsets"][4]

hidden_pts = []
n = len(sample_list)
for item in range(3, n):
    list_combinations = list(combinations(sample_list, item))
    for group in list_combinations:
        group_time_ints = numpy.diff(group).tolist()
        hidden_type = classify_by_offset(group_time_ints)
        if 'PEN' in hidden_type:
            print(group)
            print(group_time_ints)
            print(hidden_type)
            hidden_pts.append(group_time_ints)
        if 'ID' in hidden_type:
            print(group)
            print(group_time_ints)
            print(hidden_type)
            hidden_pts.append(group_time_ints)
        

list_combinations

In [ ]:
def classify_entries_as_presentation_types(piece):
    # Classifier with Functions
    points = pd.DataFrame()
    points2 = pd.DataFrame()
    # new_offset_list = []
    nr = piece.getNoteRest()
    det = piece.detailIndex(nr, offset=True)

    # durations and ngrams of durations
    dur = piece.getDuration(df=nr)
    dur_ng = piece.getNgrams(df=dur, n=4)

    # ngrams of melodic entries
    # for chromatic, use:
    # piece.getMelodicEntries(interval_settings=('c', True, True), n=5)
    mel = piece.getMelodicEntries(n=4)
    mels_stacked = mel.stack().to_frame()
    mels_stacked.rename(columns =  {0:"pattern"}, inplace = True)

    # edit distance, based on side-by-side comparison of melodic ngrams
    # gets flexed and other similar soggetti
    dist = piece.getDistance(mel)
    dist_stack = dist.stack().to_frame()


    # filter distances to threshold.  <2 is good
    filtered_dist_stack = dist_stack[dist_stack[0] < 2]
    filtered_dist = filtered_dist_stack.reset_index()
    filtered_dist.rename(columns =  {'level_0':"source", 'level_1':'match'}, inplace = True)

    # Group the filtered distanced patterns
    full_list_of_matches = filtered_dist.groupby('source')['match'].apply(list).reset_index()

    for matches in full_list_of_matches["match"]:
        related_entry_list = mels_stacked[mels_stacked['pattern'].isin(matches)]
        entry_array = related_entry_list.reset_index(level=1).rename(columns = {'level_1': "voice", 0: "pattern"})
        offset_list = entry_array.index.to_list()
        split_list = list(split_by_threshold(offset_list))
        # here is the list of starting offsets of the original set of entries:  slist
        slist = split_list[0]
        temp = temp_dict_of_details(slist, entry_array, det, matches)

        points = points.append(temp, ignore_index=True)
        points['Presentation_Type'] = points['Time_Entry_Intervals'].apply(classify_by_offset)
        points.drop_duplicates(subset=["First_Offset"], keep='first', inplace = True)
        points = points[points['Offsets'].apply(len) > 1]

        l = len(slist)
        if l > 2:
            for r in range(3, l):
    #             list_combinations = list(combinations(slist, r))
                list_combinations = list(combinations(slist, r))
                for slist in list_combinations:

                    temp = temp_dict_of_details(slist, entry_array, det, matches)

                    temp["Presentation_Type"] = classify_by_offset(temp['Time_Entry_Intervals'])

                    if 'PEN' in temp["Presentation_Type"]:
                        points2 = points2.append(temp, ignore_index=True)#.sort_values("First_Offset")
    #                     points = points.append(combo_temp, ignore_index=True).sort_values("First_Offset")
                        points2 = points2[points2['Offsets'].apply(len) > 1]
                    if 'ID' in temp["Presentation_Type"]:
                        points2 = points2.append(combo_temp, ignore_index=True)#.sort_values("First_Offset")
    #                     points = points.append(combo_temp, ignore_index=True).sort_values("First_Offset")
                points2.sort_values("First_Offset")
                points2.drop_duplicates(subset=["First_Offset"], keep='first', inplace = True)

    points_combined = points.append(points2, ignore_index=True).sort_values("First_Offset").reset_index(drop=True)
    points_combined['Flexed_Entries'] = points_combined["Soggetti"].apply(len) > 1
    points_combined["Number_Entries"] = points["Offsets"].apply(len)     
    return points2


In [ ]:
# This test works


l = len(split_list[0])  
for item in range(3, l):
    list_combinations = list(combinations(sample_list, item))
    for group in list_combinations:
        group_time_ints = numpy.diff(group).tolist()
        hidden_type = classify_by_offset(group_time_ints)
        for item in group:
#         print(item)
        array = group[entry_array.index.get_level_values(0).isin(item)]
        short_offset_list = array.index.to_list()
        time_ints = numpy.diff(array.index).tolist()
        voice_list = array['voice'].to_list()
        if 'PEN' in hidden_type:
            print(group)
            print(group_time_ints)
            print(hidden_type)
            hidden_pts.append(group_time_ints)
        if 'ID' in hidden_type:
            print(group)
            print(group_time_ints)
            print(hidden_type)
            hidden_pts.append(group_time_ints)
# len(split_list[0])           